In [48]:
# Improving Accuracy of Titanic Project

# Strategy : ignore Name, Ticket, Cabin Columns and add data scaling and pca step

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

train_df = pd.read_csv("C:/Users/user/Desktop/Machine Learning Practice/Titanic/train.csv")
test_df = pd.read_csv("C:/Users/user/Desktop/Machine Learning Practice/Titanic/test.csv")
submission_df = pd.read_csv("C:/Users/user/Desktop/Machine Learning Practice/Titanic/gender_submission.csv")

train_length = len(train_df)

df = pd.concat([train_df, test_df], ignore_index = True)

del df['Cabin']
del df['Ticket']
del df['Name']
del df['PassengerId']

df['Child'] = df['Age'] < 17
df['Adult'] = df['Age'] >= 17
df['Child'].replace(True, 1, inplace = True)
df['Child'].replace(False, 0, inplace = True)
df['Adult'].replace(True, 1, inplace = True)
df['Adult'].replace(False, 0, inplace = True)

# Filling NaN data
df['Age'].fillna(df.groupby(['Pclass', 'SibSp'])['Age'].transform('mean'), inplace = True)
df['Embarked'].fillna('S', inplace = True)
df['Fare'].fillna(df.groupby('Pclass')['Fare'].transform('mean'), inplace = True)

def merge_and_get(ldf, rdf, on, how = "inner", index = None):
    if index is True:
        return pd.merge(ldf, rdf, how = how, left_index = True, right_index = True)
    else:
        return pd.merge(ldf, rdf, how = how, on = on)

# log scale conversion
df = merge_and_get(df, np.log10(df['Fare'] + 0.5), on = None, index = True)
df.rename(columns = {'Fare_x' : 'Fare', 'Fare_y' : 'log_fare'}, inplace = True)

del df['Fare']

# make dummies for categorize

one_hot_df = merge_and_get(df, pd.get_dummies(df['Sex'], prefix = 'Sex'), on = None, index = True)
one_hot_df = merge_and_get(one_hot_df, pd.get_dummies(df['Embarked'], prefix = 'Embarked'), on = None, index = True)

del one_hot_df['Sex']
del one_hot_df['Embarked']

X_train = one_hot_df[:train_length][one_hot_df.columns[1:]] 
y_train = one_hot_df[:train_length][one_hot_df.columns[0]]
X_test = one_hot_df[train_length:][one_hot_df.columns[1:]] 
y_test = one_hot_df[train_length:][one_hot_df.columns[0]]

X_train

,Pclass,Age,SibSp,Parch,Child,Adult,log_fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,0,1,0.889302,0,1,0,0,1
1,1,38.000000,1,0,0,1,1.856023,1,0,1,0,0
2,3,26.000000,0,0,0,1,0.925570,1,0,0,0,1
3,1,35.000000,1,0,0,1,1.729165,1,0,0,0,1
4,3,35.000000,0,0,0,1,0.931966,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,0,1,1.130334,0,1,0,0,1
887,1,19.000000,0,0,0,1,1.484300,1,0,0,0,1
888,3,23.576044,1,2,0,0,1.379306,1,0,0,0,1
889,1,26.000000,0,0,0,1,1.484300,0,1,1,0,0


In [49]:
X_dev = X_train
y_dev = y_train

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size= 0.2)

In [51]:
X_train

,Pclass,Age,SibSp,Parch,Child,Adult,log_fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
413,2,31.104398,0,0,0,0,-0.301030,0,1,0,0,1
738,3,27.055311,0,0,0,0,0.924062,0,1,0,0,1
47,3,27.055311,0,0,0,0,0.916454,1,0,0,1,0
613,3,27.055311,0,0,0,0,0.916454,0,1,0,1,0
90,3,29.000000,0,0,0,1,0.931966,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
181,2,31.104398,0,0,0,0,1.191730,0,1,1,0,0
830,3,15.000000,1,0,1,0,1.174763,1,0,1,0,0
890,3,32.000000,0,0,0,1,0.916454,0,1,0,1,0
91,3,20.000000,0,0,0,1,0.921905,0,1,0,0,1


In [52]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

select = RFE(RandomForestClassifier(n_estimators = 100))

select.fit(X_train, y_train)

X_train_selected = select.transform(X_train)

In [70]:
clf = RandomForestClassifier(n_estimators = 100, max_depth = 20, random_state = 0)
clf.fit(X_train, y_train)

clf.score(X_val, y_val)

0.7877094972067039

위의 의미는 내가 수행한 data preprocessing을 통해서 얻은 모델의 점수가 78.77점일 거라는 예상이다.

In [71]:
clf.fit(X_train_selected, y_train)

clf.score(select.transform(X_val), y_val)

0.8268156424581006

In [57]:
y_test = clf.predict(select.transform(X_test))

In [58]:
y_test

array([0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 1., 1., 0.

In [65]:
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators=100, max_depth= 20, random_state = 0)

cross_val_score(clf, X_dev, y_dev, cv = 20).mean()

0.8058838383838385

In [68]:
select.fit(X_dev, y_dev)
cross_val_score(clf, select.transform(X_dev), y_dev, cv=20).mean()

0.8172727272727272

,Pclass,Age,SibSp,Parch,Child,Adult,log_fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.000000,1,0,0,1,0.889302,0,1,0,0,1
1,1,38.000000,1,0,0,1,1.856023,1,0,1,0,0
2,3,26.000000,0,0,0,1,0.925570,1,0,0,0,1
3,1,35.000000,1,0,0,1,1.729165,1,0,0,0,1
4,3,35.000000,0,0,0,1,0.931966,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,0,1,1.130334,0,1,0,0,1
887,1,19.000000,0,0,0,1,1.484300,1,0,0,0,1
888,3,23.576044,1,2,0,0,1.379306,1,0,0,0,1
889,1,26.000000,0,0,0,1,1.484300,0,1,1,0,0


In [61]:
y_test = y_test.astype(int)

y_result = pd.DataFrame(y_test)

y_result.columns = ["Survived"]

temp = submission_df['PassengerId']

submission_file = pd.concat([temp, y_result], axis = 1)

submission_file.to_csv("C:/Users/user/Desktop/Machine Learning Practice/Titanic/submit211225_decissiontree.csv", index = False)